In [ ]:
!rm -rf rgb-images
!git clone https://gbagolin@bitbucket.org/gbagolin/rgb-images.git

Cloning into 'rgb-images'...
remote: Counting objects: 365, done.
remote: Compressing objects: 100% (364/364), done.
remote: Total 365 (delta 0), reused 362 (delta 0)
Receiving objects: 100% (365/365), 78.16 MiB | 14.40 MiB/s, done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install PyTorch
!pip install -U torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
# Install MMCV
!pip install mmcv-full==latest+torch1.5.0+cu101 -f https://download.openmmlab.com/mmcv/dist/index.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 703.8MB 26kB/s 
     |████████████████████████████████| 6.6MB 46.2MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.5.0+cu101 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101
Looking in links: https://download.openmmlab.com/mmcv/dist/index.html
     |████████████████████████████████| 24.4MB 129kB/s 
     |████████████████████████████████| 194kB 4.1MB/s 


In [ ]:
!rm -rf mmsegmentation
!git clone https://github.com/open-mmlab/mmsegmentation.git 
%cd mmsegmentation
!pip install -e .

Cloning into 'mmsegmentation'...
remote: Enumerating objects: 2295, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 2295 (delta 65), reused 112 (delta 50), pack-reused 2130
Receiving objects: 100% (2295/2295), 4.39 MiB | 15.12 MiB/s, done.
Resolving deltas: 100% (1584/1584), done.
/content/mmsegmentation
Obtaining file:///content/mmsegmentation
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=b0681b594ef19daaa5d66e2507b357d3f8eb8b32e4567eed5d099eac07d45f45
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables
  Running setup.py develop for mmsegmentation


In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.5.0+cu101 True
0.12.0


## Run Inference with MMSeg trained weight

In [ ]:
!mkdir checkpoints
%cd ..
!git clone https://github.com/gbagolin/csv_dnn_creation.git

/content
Cloning into 'csv_dnn_creation'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 31 (delta 16), reused 30 (delta 15), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [ ]:
! unzip /content/drive/MyDrive/dataset/QUCTc6BB5sX_level_0.zip -d /content/

In [ ]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

In [ ]:
from os import listdir

PATH_TO_IMAGE_TO_SEGMENT = "/content/QUCTc6BB5sX_level_0/RGB/-00.6754_-02.3301_-11.9780_000.0000_000.0000_000.0000.png"
PATH_TO_CSV = "/content/csv_dnn_creation/csv/"
PATH_TO_CHECKPOINTS = "/content/mmsegmentation/checkpoints/"
PATH_TO_CONFIGS = "/content/mmsegmentation/"
IMG_NAME = '-00.6754_-02.3301_-11.9780_000.0000_000.0000_000.0000.png'
ZIP_NAME = '/content/img_out_-06.7695_001.9261_-05.0031_000.0000_120.0000_000.0000.zip'
PATH_TO_IMG_OUT = '/content/img_out/'
csv_list = listdir(PATH_TO_CSV)

In [ ]:
import pandas as pd
from os import path

for csv in csv_list:

  print(f"Model: {csv}")
  data = pd.read_csv(PATH_TO_CSV + csv)

  for index, row in data.iterrows():
    must_down = True
    checkpoint_file = PATH_TO_CHECKPOINTS + row["checkpoint_file"]
    if path.exists(checkpoint_file): 
      must_down = False

    print(f"Iter n.{index + 1}")

    link = row["checkpoint_link"]
    #this might not work 
    if must_down: 
      !wget $link -P $PATH_TO_CHECKPOINTS
    
    config_file = PATH_TO_CONFIGS + row["config_file"]
    net_name = config_file.split('/')[::-1][0][:-3]
    print(net_name)
    
    print("Creation of model...")
    # build the model from a config file and a checkpoint file
    try: 
      model = init_segmentor(config_file, checkpoint_file, device='cuda:0')
    except: 
      continue

    print("Inferencing image...")
    result = inference_segmentor(model,PATH_TO_IMAGE_TO_SEGMENT)
    out_file_name = PATH_TO_IMG_OUT + net_name + "_" + IMG_NAME
    #saving image to out_file_name path
    model.show_result(PATH_TO_IMAGE_TO_SEGMENT, result, out_file=out_file_name, opacity=1)

Model: upernet.csv
Iter n.1
upernet_r50_512x512_80k_ade20k
Creation of model...
Use load_from_local loader
Inferencing image...
Iter n.2
--2021-04-19 10:08:42--  https://download.openmmlab.com/mmsegmentation/v0.5/upernet/upernet_r101_512x512_80k_ade20k/upernet_r101_512x512_80k_ade20k_20200614_185117-32e4db94.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.75.20.25
Connecting to download.openmmlab.com (download.openmmlab.com)|47.75.20.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 342646016 (327M) [application/octet-stream]
Saving to: ‘/content/mmsegmentation/checkpoints/upernet_r101_512x512_80k_ade20k_20200614_185117-32e4db94.pth’

upernet_r101_512x51 100%[===================>] 326.77M  10.7MB/s    in 30s     

2021-04-19 10:09:13 (10.8 MB/s) - ‘/content/mmsegmentation/checkpoints/upernet_r101_512x512_80k_ade20k_20200614_185117-32e4db94.pth’ saved [342646016/342646016]

upernet_r101_512x512_80k_ade20k
Creation of model...
Use load_from_

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Iter n.2
--2021-04-19 10:10:22--  https://download.openmmlab.com/mmsegmentation/v0.5/apcnet/apcnet_r101-d8_512x512_80k_ade20k/apcnet_r101-d8_512x512_80k_ade20k_20201214_115704-c656c3fb.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.75.20.25
Connecting to download.openmmlab.com (download.openmmlab.com)|47.75.20.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 302464673 (288M) [application/octet-stream]
Saving to: ‘/content/mmsegmentation/checkpoints/apcnet_r101-d8_512x512_80k_ade20k_20201214_115704-c656c3fb.pth’

apcnet_r101-d8_512x 100%[===================>] 288.45M  10.7MB/s    in 26s     

2021-04-19 10:10:49 (11.0 MB/s) - ‘/content/mmsegmentation/checkpoints/apcnet_r101-d8_512x512_80k_ade20k_20201214_115704-c656c3fb.pth’ saved [302464673/302464673]

apcnet_r101-d8_512x512_80k_ade20k
Creation of model...
Use load_from_local loader
Inferencing image...
Iter n.3
--2021-04-19 10:10:52--  https://download.openmmlab.com/mmsegmentation/v0.5/

In [ ]:
!cp $PATH_TO_IMAGE_TO_SEGMENT $PATH_TO_IMG_OUT

In [ ]:
!zip -r $ZIP_NAME $PATH_TO_IMG_OUT

  adding: content/img_out/ (stored 0%)
  adding: content/img_out/dnl_r101-d8_512x512_160k_ade20k_-00.6754_-02.3301_-11.9780_000.0000_000.0000_000.0000.png (deflated 56%)
  adding: content/img_out/danet_r50-d8_512x512_80k_ade20k_-00.6754_-02.3301_-11.9780_000.0000_000.0000_000.0000.png (deflated 48%)
  adding: content/img_out/danet_r101-d8_512x512_80k_ade20k_-00.6754_-02.3301_-11.9780_000.0000_000.0000_000.0000.png (deflated 48%)
  adding: content/img_out/ocrnet_hr18_512x512_80k_ade20k_-00.6754_-02.3301_-11.9780_000.0000_000.0000_000.0000.png (deflated 59%)
  adding: content/img_out/deeplabv3_s101-d8_512x512_160k_ade20k_-00.6754_-02.3301_-11.9780_000.0000_000.0000_000.0000.png (deflated 56%)
  adding: content/img_out/gcnet_r101-d8_512x512_160k_ade20k_-00.6754_-02.3301_-11.9780_000.0000_000.0000_000.0000.png (deflated 58%)
  adding: content/img_out/gcnet_r50-d8_512x512_80k_ade20k_-00.6754_-02.3301_-11.9780_000.0000_000.0000_000.0000.png (deflated 49%)
  adding: content/img_out/psanet_r10

In [ ]:
rm -rf $PATH_TO_IMG_OUT

In [ ]:
rm -rf $ZIP_NAME